# Restoring a multiple hidden layer Neural Network 


![deep_network_model](../images/deep_network_model.png)


The progress of the model can be saved during and after training. This means that a model can be resumed where it left off and avoid long training times. Saving also means that you can share your model and others can recreate your work.

We will illustrate how to restore a multiple fully connected hidden layer NN. This NN have been saved in another place. We will restore the model from the "model_path" and make predictions with this so trained model after reload it.

We will use the "Concrete Compressive Strength Data Set" from:
https://archive.ics.uci.edu/ml/datasets/Concrete+Compressive+Strength

We will build a three-hidden layer neural network  to predict the fourth attribute, Petal Width from the other three (Sepal length, Sepal width, Petal length).

## Load configuration

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.datasets import load_iris
from tensorflow.python.framework import ops
import pandas as pd

/home/parrondo/anaconda3/envs/deeptrading/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/parrondo/anaconda3/envs/deeptrading/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/parrondo/anaconda3/envs/deeptrading/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/parrondo/anaconda3/envs/deeptrading/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Ingest raw data

In [2]:
#
# Leave in blanck intentionally
#

## Basic pre-process data

In [3]:
#
# Leave in blanck intentionally
#

## Split data

In [4]:
#
# Leave in blanck intentionally
#

## Transform features

In [5]:
#
# Leave in blanck intentionally
#
# If you have used data transformation becareful to ingest the transformed data to the model before
# making predictions and back-transform the predicitions. See notebook 05.
#

## Implement the model

In [6]:
# Clears the default graph stack and resets the global default graph
ops.reset_default_graph()

In [7]:
# make results reproducible
seed = 2
tf.set_random_seed(seed)
np.random.seed(seed)  


# Parameters
learning_rate = 0.005
batch_size = 50
n_features = 8#  Number of features in training data
epochs = 10000
display_step = 100
model_path = "../model/tmp/model.ckpt"
n_classes = 1

# Network Parameters
# See figure of the model
d0 = D = n_features # Layer 0 (Input layer number of features)
d1 = 5 # Layer 1 (5 hidden nodes)
d2 = 15 # Layer 2 (15 hidden nodes) 
d3 = 5 # Layer 3 (5 hidden nodes)
d4 = C = 1 # Layer 4 (Output layer)

# tf Graph input
print("Placeholders")
X = tf.placeholder(dtype=tf.float32, shape=[None, n_features], name="X")
y = tf.placeholder(dtype=tf.float32, shape=[None,n_classes], name="y")


# Initializers
print("Initializers")
sigma = 1
weight_initializer = tf.variance_scaling_initializer(mode="fan_avg", distribution="uniform", scale=sigma)
bias_initializer = tf.zeros_initializer()

# Create model
def multilayer_perceptron(X, variables):
    # Hidden layer with ReLU activation
    layer_1 = tf.nn.relu(tf.add(tf.matmul(X, variables['W1']), variables['bias1']))
    # Hidden layer with ReLU activation
    layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, variables['W2']), variables['bias2']))
    # Hidden layer with ReLU activation
    layer_3 = tf.nn.relu(tf.add(tf.matmul(layer_2, variables['W3']), variables['bias3']))
    # Output layer with ReLU activation
    out_layer = tf.nn.relu(tf.add(tf.matmul(layer_3, variables['W4']), variables['bias4']))
    return out_layer

# Store layers weight & bias
variables = {
    'W1': tf.Variable(weight_initializer([n_features, d1]), name="W1"), # inputs -> d1 hidden neurons
    'bias1': tf.Variable(bias_initializer([d1]), name="bias1"), # one biases for each d1 hidden neurons
    'W2': tf.Variable(weight_initializer([d1, d2]), name="W2"), # d1 hidden inputs -> d2 hidden neurons
    'bias2': tf.Variable(bias_initializer([d2]), name="bias2"), # one biases for each d2 hidden neurons
    'W3': tf.Variable(weight_initializer([d2, d3]), name="W3"), ## d2 hidden inputs -> d3 hidden neurons
    'bias3': tf.Variable(bias_initializer([d3]), name="bias3"), # one biases for each d3 hidden neurons
    'W4': tf.Variable(weight_initializer([d3, d4]), name="W4"), # d3 hidden inputs -> 1 output
    'bias4': tf.Variable(bias_initializer([d4]), name="bias4") # 1 bias for the output
}

# Construct model
y_hat = multilayer_perceptron(X, variables)

# Define loss and optimizer
loss = tf.reduce_mean(tf.square(y - y_hat)) # MSE
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss) # Train step

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# 'Saver' op to save and restore all the variables
saver = tf.train.Saver()

Placeholders
Initializers


## Train the model  and Evaluate the model

In [8]:
#
# Leave in blanck intentionally
#

### Saving a Tensorflow model

So, now we have our model saved.

Tensorflow model has four main files:
* a) Meta graph:
This is a protocol buffer which saves the complete Tensorflow graph; i.e. all variables, operations, collections etc. This file has .meta extension.


* b) y c) Checkpoint files:
It is a binary file which contains all the values of the weights, biases, gradients and all the other variables saved. Tensorflow has changed from version 0.11. Instead of a single .ckpt file, we have now two files: .index and .data file that contains our training variables. 


* d) Along with this, Tensorflow also has a file named checkpoint which simply keeps a record of latest checkpoint files saved.

## Predict

Finally, we can use the model to make some predictions. First we transform our samples accordingly.

In [9]:
#
# You need notebook 05 to make this transform there.
#sc.transform([[203.5, 305.3, 0.0, 203.5, 0.0, 963.4, 630.0, 90],
#              [173.0, 116.0, 0.0, 192.0, 0.0, 946.8, 856.8, 90],
#              [522.0, 0.0, 0.0, 146.0, 0.0, 896.0, 896.0, 7]]) #True value  51.86, 32.10, 50.51

Make the predictions

In [10]:
# Running a new session for predictions
print("Starting prediction session...")
with tf.Session() as sess:
    # Initialize variables
    sess.run(init)

    # Restore model weights from previously saved model
    saver.restore(sess, model_path)
    print("Model restored from file: %s" % model_path)

    # We try to predict the Concrete compressive strength (MPa megapascals) of three samples
    feed_dict_std = {X: [[ 9.78564946, 15.74026644, -2.11773518,  9.78564946, -2.11773518,
        54.23470101, 34.73303791,  3.14666097],
       [ 8.00160409,  4.66748653, -2.11773518,  9.11297662, -2.11773518,
        53.26371238, 47.99931622,  3.14666097],
       [28.41576252, -2.11773518, -2.11773518,  6.42228525, -2.11773518,
        50.29225322, 50.29225322, -1.70828215]]}
    prediction = sess.run(y_hat, feed_dict_std)
    print(prediction) #True value  51.86, 32.10, 50.51


Starting prediction session...
INFO:tensorflow:Restoring parameters from ../model/tmp/model.ckpt
Model restored from file: ../model/tmp/model.ckpt
[[0.137997 ]
 [0.137997 ]
 [1.1572909]]


In [12]:
#
# Revert transformation. Again you need notebook 05.
#y_hat_rev = sc.inverse_transform(prediction)
#y_hat_rev
#

OK, we have restored and made some predictions. We have used a model trained in other place but saved in "model_path".